In [1]:
from __future__ import print_function

import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix

NGRAMS = 2
SAMPLE = 1000000
EPOCHS = 15

# Florida voter
df = pd.read_csv('../data/fl_voter_reg/fl_reg_name_race.csv', nrows=SAMPLE)
df.dropna(subset=['name_first', 'name_last'], inplace=True)
sdf = df[df.race.isin(['multi_racial', 'native_indian']) == False].copy()

# Additional features
sdf['name_first'] = sdf.name_first.str.title()

sdf.groupby('race').agg({'name_first': 'count'})

,name_first
race,
asian,22634
hispanic,92201
nh_black,142927
nh_white,730044


## Preprocessing the input data

In [2]:
# last name only
sdf['name_last_name_first'] = sdf['name_last']

# build n-gram list
vect = CountVectorizer(analyzer='char', max_df=0.3, min_df=3, ngram_range=(NGRAMS, NGRAMS), lowercase=False) 
a = vect.fit_transform(sdf.name_last_name_first)
vocab = vect.vocabulary_

# sort n-gram by freq (highest -> lowest)
words = []
for b in vocab:
    c = vocab[b]
    #print(b, c, a[:, c].sum())
    words.append((a[:, c].sum(), b))
    #break
words = sorted(words, reverse=True)
words_list = [w[1] for w in words]
num_words = len(words_list)
print("num_words = %d" % num_words)


def find_ngrams(text, n):
    a = zip(*[text[i:] for i in range(n)])
    wi = []
    for i in a:
        w = ''.join(i)
        try:
            idx = words_list.index(w)
        except:
            idx = 0
        wi.append(idx)
    return wi

# build X from index of n-gram sequence
X = np.array(sdf.name_last_name_first.apply(lambda c: find_ngrams(c, NGRAMS)))

# check max/avg feature
X_len = []
for x in X:
    X_len.append(len(x))

max_feature_len = max(X_len)
avg_feature_len = int(np.mean(X_len))

print("Max feature len = %d, Avg. feature len = %d" % (max_feature_len, avg_feature_len))
y = np.array(sdf.race.astype('category').cat.codes)

# Split train and test dataset
X_train,  X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21, stratify=y)

num_words = 1792
Max feature len = 27, Avg. feature len = 5


## Train a LSTM model

ref: http://machinelearningmastery.com/sequence-classification-lstm-recurrent-neural-networks-python-keras/

In [3]:
'''The dataset is actually too small for LSTM to be of any advantage
compared to simpler, much faster methods such as TF-IDF + LogReg.
Notes:

- RNNs are tricky. Choice of batch size is important,
choice of loss and optimizer is critical, etc.
Some configurations won't converge.

- LSTM loss decrease patterns during training can be quite different
from what you see with CNNs/MLPs/etc.
'''
import keras
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding, Dropout, Activation
from keras.layers import LSTM
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.models import load_model

max_features = num_words # 20000
feature_len = 20 # avg_feature_len # cut texts after this number of words (among top max_features most common words)
batch_size = 32

print(len(X_train), 'train sequences')
print(len(X_test), 'test sequences')

print('Pad sequences (samples x time)')
X_train = sequence.pad_sequences(X_train, maxlen=feature_len)
X_test = sequence.pad_sequences(X_test, maxlen=feature_len)
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)

num_classes = np.max(y_train) + 1
print(num_classes, 'classes')

print('Convert class vector to binary class matrix '
      '(for use with categorical_crossentropy)')
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

/opt/venv/local/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


790244 train sequences
197562 test sequences
Pad sequences (samples x time)
X_train shape: (790244, 20)
X_test shape: (197562, 20)
4 classes
Convert class vector to binary class matrix (for use with categorical_crossentropy)
y_train shape: (790244, 4)
y_test shape: (197562, 4)


In [4]:
print('Build model...')

if True:
    model = Sequential()
    model.add(Embedding(num_words, 32, input_length=feature_len))
    model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(num_classes, activation='softmax'))
if False:
    embedding_vecor_length = 32
    model = Sequential()
    model.add(Embedding(num_words, embedding_vecor_length, input_length=feature_len))
    model.add(Conv1D(activation="relu", padding="same", filters=32, kernel_size=3))
    model.add(MaxPooling1D(pool_size=2))
    model.add(LSTM(100))
    model.add(Dense(num_classes, activation='softmax'))

# try using different optimizers and different optimizer configs
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print(model.summary())

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 32)            57344     
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               82432     
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 516       
Total params: 140,292
Trainable params: 140,292
Non-trainable params: 0
_________________________________________________________________
None


In [5]:
print('Train...')
model.fit(X_train, y_train, batch_size=batch_size, epochs=EPOCHS,
          validation_split=0.1, verbose=2)
score, acc = model.evaluate(X_test, y_test,
                            batch_size=batch_size, verbose=2)
print('Test score:', score)
print('Test accuracy:', acc)

Train...
Train on 711219 samples, validate on 79025 samples
Epoch 1/15
 - 1572s - loss: 0.6168 - acc: 0.7797 - val_loss: 0.5883 - val_acc: 0.7889
Epoch 2/15
 - 1443s - loss: 0.5793 - acc: 0.7912 - val_loss: 0.5689 - val_acc: 0.7927
Epoch 3/15
 - 1178s - loss: 0.5640 - acc: 0.7956 - val_loss: 0.5587 - val_acc: 0.7970
Epoch 4/15
 - 916s - loss: 0.5547 - acc: 0.7985 - val_loss: 0.5528 - val_acc: 0.7997
Epoch 5/15
 - 885s - loss: 0.5488 - acc: 0.8004 - val_loss: 0.5471 - val_acc: 0.8020
Epoch 6/15
 - 878s - loss: 0.5447 - acc: 0.8013 - val_loss: 0.5446 - val_acc: 0.8034
Epoch 7/15
 - 918s - loss: 0.5420 - acc: 0.8023 - val_loss: 0.5417 - val_acc: 0.8031
Epoch 8/15
 - 904s - loss: 0.5400 - acc: 0.8027 - val_loss: 0.5418 - val_acc: 0.8044
Epoch 9/15
 - 897s - loss: 0.5384 - acc: 0.8031 - val_loss: 0.5401 - val_acc: 0.8040
Epoch 10/15
 - 895s - loss: 0.5365 - acc: 0.8039 - val_loss: 0.5379 - val_acc: 0.8058
Epoch 11/15
 - 881s - loss: 0.5360 - acc: 0.8037 - val_loss: 0.5373 - val_acc: 0.8057


In [6]:
print('Test score:', score)
print('Test accuracy:', acc)

Test score: 0.533721613987878
Test accuracy: 0.807371863007076


## Confusion Matrix

In [7]:
y_pred = model.predict_classes(X_test, verbose=2)
p = model.predict_proba(X_test, verbose=2) # to predict probability
target_names = list(sdf.race.astype('category').cat.categories)
print(classification_report(np.argmax(y_test, axis=1), y_pred, target_names=target_names))
print(confusion_matrix(np.argmax(y_test, axis=1), y_pred))

             precision    recall  f1-score   support

      asian       0.77      0.41      0.54      4527
   hispanic       0.74      0.69      0.71     18440
   nh_black       0.62      0.24      0.35     28586
   nh_white       0.83      0.94      0.88    146009

avg / total       0.79      0.81      0.78    197562

[[  1860    338    105   2224]
 [    77  12758    209   5396]
 [   135    362   6927  21162]
 [   332   3836   3880 137961]]


## Save model

In [8]:
model.save('./models/fl_voter_reg/lstm/fl_all_ln_lstm.h5')

In [9]:
words_df = pd.DataFrame(words_list, columns=['vocab'])
words_df.to_csv('./models/fl_voter_reg/lstm/fl_all_ln_vocab.csv', index=False, encoding='utf-8')